In [0]:
dbutils.fs.ls("/mnt/shelldemo/")

Out[119]: [FileInfo(path='dbfs:/mnt/shelldemo/2024-08-01/', name='2024-08-01/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/shelldemo/2024-08-02/', name='2024-08-02/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/shelldemo/2024-08-03/', name='2024-08-03/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/shelldemo/2024-08-04/', name='2024-08-04/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/shelldemo/2024-08-05/', name='2024-08-05/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/shelldemo/2024-08-06/', name='2024-08-06/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/shelldemo/2024-08-07/', name='2024-08-07/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/shelldemo/PreviousData/', name='PreviousData/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/shelldemo/merged/', name='merged/', size=0, modificationTime=0)]

In [0]:
folders = dbutils.fs.ls("/mnt/shelldemo/")
dates = [folder.name.strip("/") for folder in folders if folder.isDir()]
dates

Out[128]: ['2024-08-01',
 '2024-08-02',
 '2024-08-03',
 '2024-08-04',
 '2024-08-05',
 '2024-08-06',
 '2024-08-07',
 'PreviousData',
 'merged']

In [0]:
dbutils.fs.ls("/mnt/shelldemo/2024-08-01/IN")

Out[124]: [FileInfo(path='dbfs:/mnt/shelldemo/2024-08-01/IN', name='IN', size=8541, modificationTime=1723145520000)]

In [0]:
from pyspark.sql.functions import input_file_name

# Initialize an empty DataFrame
merged_df = None

# Iterate over each date and read the corresponding files
for date in dates:
    for country in ["IN"]:
        file_path1 = f"/mnt/shelldemo/{date}/{country}"
        df = spark.read.option("header", "true").csv(file_path1)
        
        if merged_df is None:
            merged_df = df
        else:
            merged_df = merged_df.unionByName(df)

# Perform any additional transformations if needed

# Save the merged DataFrame to a single file
output_path = "/mnt/shelldemo/merged/merged_csv.csv"
merged_df.write.format("delta").mode("overwrite").save(output_path)

display(merged_df)
# Show the merged Delta table
#merged_df.show()

In [0]:
merged_df.printSchema

Out[131]: <bound method DataFrame.printSchema of DataFrame[Country_ID: string, Date: string, Site_ID: string, Product_ID: string, Volume_Sold: string]>

India File

In [0]:
from pyspark.sql.functions import when,trim,col, to_date, current_timestamp

merged_IN = (merged_df.withColumn("Country_ID", when(col("Country_ID") == "IND", "IN").otherwise(col("Country_ID")))
                      .withColumn("Site_ID", when(col("Site_ID") == "null", None).otherwise(col("Site_ID")))
                      .withColumn("Product_ID", when(col("Product_ID") == "null", None).otherwise(col("Product_ID")))
                      .withColumn("Volume_Sold", when(col("Volume_Sold") == "null", None).otherwise(col("Volume_Sold")))
                      .filter(col("Site_ID").isNotNull() & col("Product_ID").isNotNull() & col("Volume_Sold").isNotNull())
                      .withColumn("Date", date_format(to_date(col("Date"), "dd/MM/yyyy"), "yyyy-MM-dd"))
                      .withColumn("LoadDateTime", current_timestamp())
            )

display(merged_IN)

Country_ID,Date,Site_ID,Product_ID,Volume_Sold,LoadDateTime
IN,2024-07-27,1,91,450.2 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,1,94,443.0 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,1,98,493.3 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,1,99,668.0 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,2,89,554.3 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,2,91,266.5 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,2,94,478.2 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,2,98,751.8 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,2,99,900.3 liters,2024-08-09T18:10:37.395+0000
IN,2024-07-27,3,89,432.3 liters,2024-08-09T18:10:37.395+0000


US File

In [0]:
from pyspark.sql.functions import input_file_name

# Initialize an empty DataFrame
merged_df = None

# Iterate over each date and read the corresponding files
for date in dates:
    for country in ["US"]:
        file_path = f"/mnt/shelldemo/{date}/{country}"
        df = spark.read.option("header", "true").csv(file_path)
        
        if merged_df is None:
            merged_df = df
        else:
            merged_df = merged_df.unionByName(df)

# Perform any additional transformations if needed

# Save the merged DataFrame to a single file
output_path = "/mnt/shelldemo/merged/merged_us.csv"
merged_df.write.format("delta").mode("overwrite").save(output_path)

display(merged_df)
# Show the merged Delta table
#merged_df.show()

In [0]:
from pyspark.sql.functions import when,trim,col, to_date, current_timestamp, regexp_extract, concat, lit, format_number

gallons_to_liters = 3.78541
merged_US = (merged_df.withColumn("Country_ID", when(col("Country_ID") == "USA", "US").otherwise(col("Country_ID")))
                      .withColumn("Site_ID", when(col("Site_ID") == "null", None).otherwise(col("Site_ID")))
                      .withColumn("Product_ID", when(col("Product_ID") == "null", None).otherwise(col("Product_ID")))
                      .withColumn("Volume_Sold", when(col("Volume_Sold") == "null", None).otherwise(col("Volume_Sold")))
                      .filter(col("Site_ID").isNotNull() & col("Product_ID").isNotNull() & col("Volume_Sold").isNotNull())
                      .withColumn("Volume_Sold_Liters", format_number(regexp_extract(col("Volume_Sold"), r"(\d+(\.\d+)?)", 1).cast("double") * gallons_to_liters, 1))
                      .withColumn("Volume_Sold",concat(col("Volume_Sold_Liters"), lit(" liters")))
                      .withColumn("Date", date_format(to_date(col("Date"), "mm/dd/yyyy"), "yyyy-MM-dd"))
                      .withColumn("LoadDateTime", current_timestamp()).drop(col("Volume_Sold_Liters"))
            )

display(merged_US)

Country_ID,Date,Site_ID,Product_ID,Volume_Sold,LoadDateTime
US,2024-01-27,1,91,535.3 liters,2024-08-09T18:10:48.803+0000
US,2024-01-27,1,94,"3,019.2 liters",2024-08-09T18:10:48.803+0000
US,2024-01-27,1,98,659.0 liters,2024-08-09T18:10:48.803+0000
US,2024-01-27,1,99,"3,221.8 liters",2024-08-09T18:10:48.803+0000
US,2024-01-27,2,91,"2,988.6 liters",2024-08-09T18:10:48.803+0000
US,2024-01-27,2,94,"1,974.5 liters",2024-08-09T18:10:48.803+0000
US,2024-01-27,2,98,"2,275.8 liters",2024-08-09T18:10:48.803+0000
US,2024-01-27,2,99,544.7 liters,2024-08-09T18:10:48.803+0000
US,2024-01-27,3,89,"3,429.6 liters",2024-08-09T18:10:48.803+0000
US,2024-01-27,3,91,731.3 liters,2024-08-09T18:10:48.803+0000


Netherlands File

In [0]:
from pyspark.sql.functions import input_file_name

# Initialize an empty DataFrame
merged_df = None

# Iterate over each date and read the corresponding files
for date in dates:
    for country in ["NL"]:
        file_path = f"/mnt/shelldemo/{date}/{country}"
        df = spark.read.option("header", "true").csv(file_path)
        
        
        if merged_df is None:
            merged_df = df
        else:
            merged_df = merged_df.unionByName(df)

# Perform any additional transformations if needed

# Save the merged DataFrame to a single file
output_path = "/mnt/shelldemo/merged/merged_nl.csv"
merged_df.write.format("delta").mode("overwrite").save(output_path)

display(merged_df)
# Show the merged Delta table
#merged_df.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2455698954992831> in <cell line: 7>()
      8     for country in ["NL"]:
      9         file_path = f"/mnt/shelldemo/{date}/{country}"
---> 10         df = spark.read.option("header", "true").csv(file_path)
     11 
     12 

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, t

In [0]:
from pyspark.sql.functions import when,trim,col, to_date, current_timestamp

merged_NL = (merged_df.withColumn("Site_ID", when(col("Site_ID") == "null", None).otherwise(col("Site_ID")))
                      .withColumn("Product_ID", when(col("Product_ID") == "null", None).otherwise(col("Product_ID")))
                      .withColumn("Volume_Sold", when(col("Volume_Sold") == "null", None).otherwise(col("Volume_Sold")))
                      .filter(col("Site_ID").isNotNull() & col("Product_ID").isNotNull() & col("Volume_Sold").isNotNull())
                      .withColumn("Date", date_format(to_date(col("Date"), "dd/MM/yyyy"), "yyyy-MM-dd"))
                      .withColumn("LoadDateTime", current_timestamp())
            )

display(merged_NL)

In [0]:
merged_df = merged_IN.unionByName(merged_US)

# Merge the result with df3
merged_df = merged_df.unionByName(merged_NL)

display(merged_df)

Country_ID,Date,Site_ID,Product_ID,Volume_Sold,LoadDateTime
IN,2024-07-27,1,91,450.2 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,1,94,443.0 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,1,98,493.3 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,1,99,668.0 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,2,89,554.3 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,2,91,266.5 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,2,94,478.2 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,2,98,751.8 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,2,99,900.3 liters,2024-08-09T19:14:08.346+0000
IN,2024-07-27,3,89,432.3 liters,2024-08-09T19:14:08.346+0000


In [0]:
merged_df.printSchema

Out[138]: <bound method DataFrame.printSchema of DataFrame[Country_ID: string, Date: string, Site_ID: string, Product_ID: string, Volume_Sold: string, LoadDateTime: timestamp]>

In [0]:
from pyspark.sql.functions import regexp_replace, col, to_date
from pyspark.sql.types import IntegerType, DoubleType, DateType
df = merged_df.withColumn("Volume_Sold_Liters", regexp_replace(col("Volume_Sold"), " liters", "")).drop(col("Volume_Sold"))

# Convert columns to the specified types
df = (df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))
        .withColumn("Site_ID", col("Site_ID").cast(IntegerType()))
        .withColumn("Product_ID", col("Product_ID").cast(IntegerType()))
        .withColumn("Volume_Sold_Liters", col("Volume_Sold_Liters").cast(DoubleType()))
     )

df = df.select(
    "Country_ID","Date","Site_ID","Product_ID","Volume_Sold_Liters","LoadDateTime"
)

display(df)

Country_ID,Date,Site_ID,Product_ID,Volume_Sold_Liters,LoadDateTime
IN,2024-07-27,1,91,450.2,2024-08-09T18:11:07.277+0000
IN,2024-07-27,1,94,443.0,2024-08-09T18:11:07.277+0000
IN,2024-07-27,1,98,493.3,2024-08-09T18:11:07.277+0000
IN,2024-07-27,1,99,668.0,2024-08-09T18:11:07.277+0000
IN,2024-07-27,2,89,554.3,2024-08-09T18:11:07.277+0000
IN,2024-07-27,2,91,266.5,2024-08-09T18:11:07.277+0000
IN,2024-07-27,2,94,478.2,2024-08-09T18:11:07.277+0000
IN,2024-07-27,2,98,751.8,2024-08-09T18:11:07.277+0000
IN,2024-07-27,2,99,900.3,2024-08-09T18:11:07.277+0000
IN,2024-07-27,3,89,432.3,2024-08-09T18:11:07.277+0000


In [0]:
df.printSchema

Out[140]: <bound method DataFrame.printSchema of DataFrame[Country_ID: string, Date: date, Site_ID: int, Product_ID: int, Volume_Sold_Liters: double, LoadDateTime: timestamp]>

Read Historical Data

In [0]:
from pyspark.sql.functions import input_file_name

# Define the file path for reading the data
file_path = "/mnt/shelldemo/PreviousData/June_July_21_24.csv"

# Read the CSV file into a DataFrame
df_history = spark.read.option("header", "true").csv(file_path)

# Define the output path for saving the data
output_path = "/mnt/shelldemo/merged/history"

# Save the DataFrame to a Delta table
df_history.write.format("delta").mode("overwrite").save(output_path)

df_history = df_history.withColumn("Volume_Sold_Liters", regexp_replace(col("Volume_Sold"), " liters", "")).drop(col("Volume_Sold"))

# Convert columns to the specified types
df_history = (df_history.withColumn("Country_ID", when(col("Country_ID") == "USA", "US")
                                    .when(col("Country_ID") == "IND", "IN")
                                    .otherwise(col("Country_ID"))
                                   )
                        .withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))
                        .withColumn("Site_ID", col("Site_ID").cast(IntegerType()))
                        .withColumn("Product_ID", col("Product_ID").cast(IntegerType()))
                        .withColumn("Volume_Sold_Liters", col("Volume_Sold_Liters").cast(DoubleType()))
                        .withColumn("LoadDateTime", current_timestamp())
     )

df_history = df_history.select(
    "Country_ID","Date","Site_ID","Product_ID","Volume_Sold_Liters","LoadDateTime"
)

display(df_history)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2455698954992840> in <cell line: 7>()
      5 
      6 # Read the CSV file into a DataFrame
----> 7 df_history = spark.read.option("header", "true").csv(file_path)
      8 
      9 # Define the output path for saving the data

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, t

In [0]:
df_history.printSchema

Out[142]: <bound method DataFrame.printSchema of DataFrame[Country_ID: string, Date: date, Site_ID: int, Product_ID: int, Volume_Sold_Liters: double, LoadDateTime: timestamp]>